In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils.config import Config
from utils.parse_dataset import NetworkDataset, load_datasets
from utils.load_models import GRU_Models

In [2]:
conf = Config()
load_models = GRU_Models()
model = load_models.gru_4

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513, 1]) torch.Size([512])


In [5]:
acc, train_loss, val_loss = model.train(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 88.24%, Train loss: 0.8653, Val loss: 0.2524
Epoch: 2/10, Accuracy: 91.85%, Train loss: 0.2446, Val loss: 0.1325
Epoch: 3/10, Accuracy: 93.88%, Train loss: 0.1515, Val loss: 0.0956
Epoch: 4/10, Accuracy: 94.67%, Train loss: 0.1133, Val loss: 0.0800
Epoch: 5/10, Accuracy: 94.79%, Train loss: 0.0956, Val loss: 0.0731
Epoch: 6/10, Accuracy: 95.68%, Train loss: 0.0856, Val loss: 0.0635
Epoch: 7/10, Accuracy: 96.18%, Train loss: 0.0777, Val loss: 0.0574
Epoch: 8/10, Accuracy: 96.34%, Train loss: 0.0729, Val loss: 0.0554
Epoch: 9/10, Accuracy: 96.27%, Train loss: 0.0688, Val loss: 0.0544
Epoch: 10/10, Accuracy: 96.44%, Train loss: 0.0658, Val loss: 0.0527


In [6]:
loss, accuracy = model.evaluate(test_loader)
print(f" Accuracy: {100*accuracy:.2f}%")

 Accuracy: 96.37%


In [7]:
model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/large_models/checkpoint/gru_4.pth
